In [3]:
import hytools as ht
import matplotlib.pyplot as plt
import numpy as np



In [5]:
envi = ht.HyTools()
envi.read_file("output_file",'envi')

In [7]:
envi.get_pixels(500,500)

memmap([230.89001118, 265.29973664, 227.50122958, 453.15689901,
        289.9373611 , 150.14576218])

In [8]:
#plt.plot(envi.wavelengths,original[0])
original = envi.get_pixels([500],[500])
plt.plot(outwaves,resample[0])
plt.ylim(0,.6)

Input wavelength outside wavelength range!


ValueError: not enough values to unpack (expected 2, got 0)